In [1]:
import pandas as pd

base = pd.read_csv('dataset.csv')

In [2]:
import numpy as np

previsores = base.iloc[:1400000,0:20].values
previsores = np.reshape(previsores, (2000, 700, 20))
print(previsores.shape)

classes = base.iloc[:1400000, 28:31].values
classes = np.reshape(classes, (2000, 700, 3))
print(classes.shape)

(2000, 700, 20)
(2000, 700, 3)


In [4]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, CuDNNLSTM, Bidirectional, Masking

In [5]:
def criarRede():
    model = Sequential()
  
    #model.add(Masking(mask_value = 0, input_shape = (700, 20)))
  
    model.add(Bidirectional(CuDNNLSTM(100, return_sequences = True), input_shape = (700, 20)))
    model.add(Dropout(0.3))
    
    model.add(Bidirectional(CuDNNLSTM(100, return_sequences = True)))
    model.add(Dropout(0.3))
  
    model.add(Dense(3, activation = 'softmax'))
  
    model.compile(optimizer = 'adam', metrics = ['acc'], loss='categorical_crossentropy')
  
    return model

In [6]:
accu = []
precisions = []
recalls = []
f1 = []

In [7]:
from sklearn.metrics import balanced_accuracy_score, classification_report, precision_score, recall_score, f1_score, confusion_matrix

def train_and_evaluate_model(model, x_train, y_train, x_test, y_test):
    model.fit(x_train, y_train, epochs = 100, verbose = 1, batch_size = 32)
  
    predicted = model.predict(x_test)
  
    y_teste = []
    predict = []
  
    predicted = np.reshape(predicted, (predicted.shape[0] * predicted.shape[1], 3))
    y_test = np.reshape(y_test, (y_test.shape[0] * y_test.shape[1], 3))
    x_test = np.reshape(x_test, (x_test.shape[0] * x_test.shape[1], 20))

    for i in range(len(x_test)):
        cont = 0
        for j in range(len(x_test[i])):
            cont += x_test[i][j]
        if cont != 0:
            y_teste.append(y_test[i])
            predict.append(predicted[i])
    
    y_teste = np.asarray(y_teste)
    predict = np.asarray(predict)

    predicted = predict
    y_test = y_teste

    predicted = np.argmax(predicted, axis=1)
    y_test = np.argmax(y_test, axis=1)

    accu.append(balanced_accuracy_score(y_test, predicted)) 
    precisions.append(precision_score(y_test, predicted, average = 'weighted'))
    recalls.append(recall_score(y_test, predicted, average = 'weighted'))
    f1.append(f1_score(y_test, predicted, average = 'weighted'))
    
    print(classification_report(y_test, predicted))
    print('Acurácia')
    print(balanced_accuracy_score(y_test, predicted))
    print('Precisao')
    print(precision_score(y_test, predicted, average = 'weighted'))
    print('Recall')
    print(recall_score(y_test, predicted, average = 'weighted'))
    print('F1')
    print(f1_score(y_test, predicted, average = 'weighted'))
    print(confusion_matrix(y_test, predicted))

In [8]:
from sklearn.model_selection import KFold

kf = KFold(n_splits = 5, shuffle = True)

for train_index, test_index in kf.split(previsores):
    model = None
    model = criarRede()
    print("TRAIN:", train_index, "TEST:", test_index)
    train_and_evaluate_model(model, previsores[train_index], classes[train_index],
                           previsores[test_index], classes[test_index])

W0819 20:18:33.910733 15276 deprecation.py:506] From c:\users\gabriel\appdata\local\programs\python\python36\lib\site-packages\tensorflow\python\ops\init_ops.py:1251: calling VarianceScaling.__init__ (from tensorflow.python.ops.init_ops) with dtype is deprecated and will be removed in a future version.
Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor
W0819 20:18:33.915692 15276 deprecation.py:506] From c:\users\gabriel\appdata\local\programs\python\python36\lib\site-packages\tensorflow\python\ops\init_ops.py:97: calling GlorotUniform.__init__ (from tensorflow.python.ops.init_ops) with dtype is deprecated and will be removed in a future version.
Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor
W0819 20:18:33.917707 15276 deprecation.py:506] From c:\users\gabriel\appdata\local\programs\python\python36\lib\site-packages\tensorflow\python\ops\init_ops.py

TRAIN: [   0    1    2 ... 1996 1998 1999] TEST: [   3    6   10   11   12   18   19   20   25   28   29   37   46   57
   58   59   63   69   74   77   85   97  100  101  102  103  108  117
  128  133  136  139  140  144  157  166  175  179  182  187  190  191
  192  198  202  210  231  233  234  237  238  248  249  259  263  269
  272  274  275  280  296  307  309  318  322  327  329  330  341  351
  354  355  362  363  370  372  373  380  382  386  387  395  403  415
  419  425  426  428  430  431  437  440  447  461  462  463  464  469
  470  482  483  491  494  495  504  513  514  530  531  533  560  565
  576  585  596  600  605  606  610  614  619  622  623  629  632  634
  641  643  651  652  654  655  659  665  671  685  689  701  706  709
  711  713  717  725  728  733  736  740  748  750  760  764  770  774
  775  787  791  796  798  800  811  813  815  818  819  822  826  831
  835  838  853  871  876  882  891  899  901  910  915  933  944  951
  953  955  967  987 1010 10

W0819 20:18:34.670674 15276 deprecation.py:323] From c:\users\gabriel\appdata\local\programs\python\python36\lib\site-packages\tensorflow\python\ops\math_grad.py:1250: add_dispatch_support.<locals>.wrapper (from tensorflow.python.ops.array_ops) is deprecated and will be removed in a future version.
Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where


Epoch 1/100
1600/1600 [==============================] - 8s 5ms/sample - loss: 0.3566 - acc: 0.7292
Epoch 2/100
1600/1600 [==============================] - 6s 3ms/sample - loss: 0.3171 - acc: 0.8499
Epoch 3/100
1600/1600 [==============================] - 6s 3ms/sample - loss: 0.3063 - acc: 0.8613
Epoch 4/100
1600/1600 [==============================] - 6s 3ms/sample - loss: 0.3023 - acc: 0.8638
Epoch 5/100
1600/1600 [==============================] - 6s 3ms/sample - loss: 0.2953 - acc: 0.8674
Epoch 6/100
1600/1600 [==============================] - 6s 3ms/sample - loss: 0.2923 - acc: 0.8692
Epoch 7/100
1600/1600 [==============================] - 6s 4ms/sample - loss: 0.2893 - acc: 0.8708
Epoch 8/100
1600/1600 [==============================] - 6s 4ms/sample - loss: 0.2895 - acc: 0.8706
Epoch 9/100
1600/1600 [==============================] - 6s 4ms/sample - loss: 0.2853 - acc: 0.8725
Epoch 10/100
1600/1600 [==============================] - 6s 4ms/sample - loss: 0.2844 - acc: 0.8728

1600/1600 [==============================] - 6s 3ms/sample - loss: 0.1557 - acc: 0.9315
Epoch 83/100
1600/1600 [==============================] - 6s 3ms/sample - loss: 0.1548 - acc: 0.9328
Epoch 84/100
1600/1600 [==============================] - 6s 3ms/sample - loss: 0.1531 - acc: 0.9335
Epoch 85/100
1600/1600 [==============================] - 6s 3ms/sample - loss: 0.1516 - acc: 0.9335
Epoch 86/100
1600/1600 [==============================] - 6s 3ms/sample - loss: 0.1519 - acc: 0.9332
Epoch 87/100
1600/1600 [==============================] - 6s 3ms/sample - loss: 0.1540 - acc: 0.9325
Epoch 88/100
1600/1600 [==============================] - 5s 3ms/sample - loss: 0.1513 - acc: 0.9333
Epoch 89/100
1600/1600 [==============================] - 6s 3ms/sample - loss: 0.1510 - acc: 0.9348
Epoch 90/100
1600/1600 [==============================] - 6s 3ms/sample - loss: 0.1480 - acc: 0.9356
Epoch 91/100
1600/1600 [==============================] - 6s 3ms/sample - loss: 0.1479 - acc: 0.9350
Epo

...

Epoch 1/100
1600/1600 [==============================] - 6s 4ms/sample - loss: 0.3613 - acc: 0.8094
Epoch 2/100
1600/1600 [==============================] - 5s 3ms/sample - loss: 0.3224 - acc: 0.8521
Epoch 3/100
1600/1600 [==============================] - 5s 3ms/sample - loss: 0.3076 - acc: 0.8609
Epoch 4/100
1600/1600 [==============================] - 5s 3ms/sample - loss: 0.3044 - acc: 0.8635
Epoch 5/100
1600/1600 [==============================] - 5s 3ms/sample - loss: 0.2997 - acc: 0.8665
Epoch 6/100
1600/1600 [==============================] - 5s 3ms/sample - loss: 0.2950 - acc: 0.8690
Epoch 7/100
1600/1600 [==============================] - 5s 3ms/sample - loss: 0.2915 - acc: 0.8708
Epoch 8/100
1600/1600 [==============================] - 5s 3ms/sample - loss: 0.2911 - acc: 0.8712
Epoch 9/100
1600/1600 [==============================] - 5s 3ms/sample - loss: 0.2886 - acc: 0.8720
Epoch 10/100
1600/1600 [==============================] - 5s 3ms/sample - loss: 0.2857 - acc: 0.8738

1600/1600 [==============================] - 5s 3ms/sample - loss: 0.1622 - acc: 0.9309
Epoch 83/100
1600/1600 [==============================] - 5s 3ms/sample - loss: 0.1613 - acc: 0.9312
Epoch 84/100
1600/1600 [==============================] - 5s 3ms/sample - loss: 0.1618 - acc: 0.9312
Epoch 85/100
1600/1600 [==============================] - 5s 3ms/sample - loss: 0.1618 - acc: 0.9309
Epoch 86/100
1600/1600 [==============================] - 5s 3ms/sample - loss: 0.1692 - acc: 0.9278
Epoch 87/100
1600/1600 [==============================] - 5s 3ms/sample - loss: 0.1616 - acc: 0.9312
Epoch 88/100
1600/1600 [==============================] - 5s 3ms/sample - loss: 0.1600 - acc: 0.9320
Epoch 89/100
1600/1600 [==============================] - 5s 3ms/sample - loss: 0.1562 - acc: 0.9336
Epoch 90/100
1600/1600 [==============================] - 5s 3ms/sample - loss: 0.1549 - acc: 0.9344
Epoch 91/100
1600/1600 [==============================] - 5s 3ms/sample - loss: 0.1537 - acc: 0.9351
Epo

1600/1600 [==============================] - 5s 3ms/sample - loss: 0.2429 - acc: 0.8939
Epoch 38/100
1600/1600 [==============================] - 5s 3ms/sample - loss: 0.2409 - acc: 0.8949
Epoch 39/100
1600/1600 [==============================] - 6s 3ms/sample - loss: 0.2410 - acc: 0.8946
Epoch 40/100
1600/1600 [==============================] - 6s 3ms/sample - loss: 0.2408 - acc: 0.8948
Epoch 41/100
1600/1600 [==============================] - 6s 3ms/sample - loss: 0.2356 - acc: 0.8972
Epoch 42/100
1600/1600 [==============================] - 5s 3ms/sample - loss: 0.2321 - acc: 0.8997
Epoch 43/100
1600/1600 [==============================] - 5s 3ms/sample - loss: 0.2296 - acc: 0.9004
Epoch 44/100
1600/1600 [==============================] - 5s 3ms/sample - loss: 0.2276 - acc: 0.9009
Epoch 45/100
1600/1600 [==============================] - 5s 3ms/sample - loss: 0.2258 - acc: 0.9024
Epoch 46/100
1600/1600 [==============================] - 6s 3ms/sample - loss: 0.2236 - acc: 0.9031
Epo

Epoch 1/100
1600/1600 [==============================] - 6s 4ms/sample - loss: 0.3605 - acc: 0.8197
Epoch 2/100
1600/1600 [==============================] - 5s 3ms/sample - loss: 0.3225 - acc: 0.8515
Epoch 3/100
1600/1600 [==============================] - 5s 3ms/sample - loss: 0.3109 - acc: 0.8599
Epoch 4/100
1600/1600 [==============================] - 5s 3ms/sample - loss: 0.3052 - acc: 0.8635
Epoch 5/100
1600/1600 [==============================] - 5s 3ms/sample - loss: 0.3020 - acc: 0.8655
Epoch 6/100
1600/1600 [==============================] - 5s 3ms/sample - loss: 0.2977 - acc: 0.8677
Epoch 7/100
1600/1600 [==============================] - 5s 3ms/sample - loss: 0.2934 - acc: 0.8697
Epoch 8/100
1600/1600 [==============================] - 5s 3ms/sample - loss: 0.2920 - acc: 0.8704
Epoch 9/100
1600/1600 [==============================] - 5s 3ms/sample - loss: 0.2904 - acc: 0.8713
Epoch 10/100
1600/1600 [==============================] - 5s 3ms/sample - loss: 0.2921 - acc: 0.8698

1600/1600 [==============================] - 5s 3ms/sample - loss: 0.1633 - acc: 0.9307
Epoch 83/100
1600/1600 [==============================] - 5s 3ms/sample - loss: 0.1636 - acc: 0.9307
Epoch 84/100
1600/1600 [==============================] - 5s 3ms/sample - loss: 0.1632 - acc: 0.9309
Epoch 85/100
1600/1600 [==============================] - 5s 3ms/sample - loss: 0.1634 - acc: 0.9308
Epoch 86/100
1600/1600 [==============================] - 6s 3ms/sample - loss: 0.1600 - acc: 0.9324
Epoch 87/100
1600/1600 [==============================] - 5s 3ms/sample - loss: 0.1573 - acc: 0.9335
Epoch 88/100
1600/1600 [==============================] - 5s 3ms/sample - loss: 0.1558 - acc: 0.9340
Epoch 89/100
1600/1600 [==============================] - 5s 3ms/sample - loss: 0.1552 - acc: 0.9344
Epoch 90/100
1600/1600 [==============================] - 6s 3ms/sample - loss: 0.1541 - acc: 0.9350
Epoch 91/100
1600/1600 [==============================] - 6s 3ms/sample - loss: 0.1531 - acc: 0.9353
Epo

In [9]:
model.summary()

Model: "sequential_4"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
bidirectional_8 (Bidirection (None, 700, 200)          97600     
_________________________________________________________________
dropout_8 (Dropout)          (None, 700, 200)          0         
_________________________________________________________________
bidirectional_9 (Bidirection (None, 700, 200)          241600    
_________________________________________________________________
dropout_9 (Dropout)          (None, 700, 200)          0         
_________________________________________________________________
dense_4 (Dense)              (None, 700, 3)            603       
Total params: 339,803
Trainable params: 339,803
Non-trainable params: 0
_________________________________________________________________


In [10]:
print('Acurácias total')
print(accu)
accu = np.array(accu)
print(accu.mean())
print('Precision total')
print(precisions)
precisions = np.array(precisions)
print(precisions.mean())
print('Recalls total')
print(recalls)
recalls = np.array(recalls)
print(recalls.mean())
print('F1 total')
print(f1)
f1 = np.array(f1)
print(f1.mean())

Acurácias total
[0.6605568753265784, 0.6692014567351116, 0.6556765832170971, 0.6673096735662384, 0.6592568789945416]
0.6624002935679134
Precision total
[0.6798332561868559, 0.6866890465517655, 0.6742711569895773, 0.6828845695061632, 0.6769864327075001]
0.6801328923883725
Recalls total
[0.6805046263750879, 0.6854131755087808, 0.6746496015075969, 0.6833576684130666, 0.6755669058171194]
0.6798983955243303
F1 total
[0.6792289140529946, 0.6838828951316156, 0.6726982394613646, 0.6815736352934462, 0.6748151429419853]
0.6784397653762813
